In [6]:
%matplotlib inline

In [7]:
import matplotlib.pylab as plt
import torch
import numpy as np
import seaborn as sn
sn.set_context("poster")

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils import weight_norm
from torchvision import transforms, datasets

from deepsith import DeepSITH

import numpy as np
import scipy
import scipy.stats as st
import scipy.special
import scipy.signal
import scipy.interpolate

import pandas as pd

from os.path import join
import random
from csv import DictWriter

from tqdm.notebook import tqdm
import pickle
# if gpu is to be used
use_cuda = torch.cuda.is_available()
print(use_cuda)
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
DoubleTensor = torch.cuda.DoubleTensor if use_cuda else torch.DoubleTensor
IntTensor = torch.cuda.IntTensor if use_cuda else torch.IntTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if use_cuda else torch.ByteTensor
ttype = FloatTensor

True


In [8]:
import torch
import numpy as np

def get_batch(batch_size, T, ttype):
    values = torch.rand(T, batch_size, requires_grad=False)
    indices = torch.zeros_like(values)
    half = int(T / 2)
    for i in range(batch_size):
        half_1 = np.random.randint(half)
        hals_2 = np.random.randint(half, T)
        indices[half_1, i] = 1
        indices[hals_2, i] = 1

    data = torch.stack((values, indices), dim=-1).type(ttype)
    targets = torch.mul(values, indices).sum(dim=0).type(ttype)
    return data, targets


In [9]:
torch.manual_seed(1111)

In [10]:
seq_length = 600

In [11]:
xtemp, ytemp = get_batch(4, seq_length, ttype=ttype)
print(xtemp.shape, ytemp.shape)

torch.Size([600, 4, 2]) torch.Size([4])


In [12]:
xtemp.transpose(0,1).transpose(1,2).unsqueeze(1)

tensor([[[[0.4606, 0.9548, 0.5145,  ..., 0.5468, 0.3088, 0.1194],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]],


        [[[0.0850, 0.5982, 0.8125,  ..., 0.1295, 0.0968, 0.9441],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]],


        [[[0.8009, 0.4821, 0.3122,  ..., 0.5019, 0.9653, 0.9626],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]],


        [[[0.3972, 0.9446, 0.9756,  ..., 0.9959, 0.1073, 0.8790],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]]],
       device='cuda:0')

In [13]:
X_train.shape

NameError: name 'X_train' is not defined

In [18]:
def train(model, ttype, seq_length, optimizer, loss_func, 
          epoch, perf_file, loss_buffer_size=20, batch_size=1, 
          device='cuda', prog_bar=None):
    assert(loss_buffer_size%batch_size==0)

    losses = []
    perfs = []
    last_test_perf = 0
    for batch_idx in range(60000):
        model.train()
        sig, target = get_batch(batch_size, seq_length, ttype=ttype)
        sig = sig.transpose(0,1).transpose(1,2).unsqueeze(1)
        target = target.unsqueeze(1)
        optimizer.zero_grad()
        out = model(sig)
        loss = loss_func(out[:, -1, :],
                         target)
         
        loss.backward()
        optimizer.step()

        losses.append(loss.detach().cpu().numpy())
        losses = losses[-loss_buffer_size:]
        if not (prog_bar is None):
            # Update progress_bar
            s = "{}:{} Loss: {:.4f}"
            format_list = [e, int(batch_idx/(50/batch_size)), np.mean(losses)]         
            s = s.format(*format_list)
            prog_bar.set_description(s)
        if ((batch_idx*batch_size)%loss_buffer_size == 0) & (batch_idx != 0):
            loss_track = {}
           # last_test_perf = test_norm(model, 'cuda', test_sig, test_class,
            #                                    batch_size=batch_size, 
            #                                    )
            loss_track['avg_loss'] = np.mean(losses)
            #loss_track['last_test'] = last_test_perf
            loss_track['epoch'] = epoch
            loss_track['batch_idx'] = batch_idx
            #loss_track['train_perf']= np.sum(perfs)/((len(perfs))*batch_size)
            with open(perf_file, 'a+') as fp:
                csv_writer = DictWriter(fp, fieldnames=list(loss_track.keys()))
                if fp.tell() == 0:
                    csv_writer.writeheader()
                csv_writer.writerow(loss_track)
                fp.flush()
def test_norm(model, device, seq_length, loss_func, batch_size=1):
    model.eval()
    correct = 0
    count = 0
    with torch.no_grad():
        out = model(test_sig)
        loss = loss_func(out[:, -1, :],
                         test_outs)
        pred = out[:, -1].argmax(dim=-1, keepdim=True)
        correct += pred.eq(test_class.view_as(pred)).sum().item()
        count += 1
    return correct / test_sig.shape[0]

In [15]:
class DeepSITH_Predictor(nn.Module):
    def __init__(self, out_features, layer_params, dropout=.5):
        super(DeepSITH_Predictor, self).__init__()
        last_hidden = layer_params[-1]['hidden_size']
        self.hs = DeepSITH(layer_params=layer_params, dropout=dropout)
        self.to_out = nn.Linear(last_hidden, out_features)
    def forward(self, inp):
        x = self.hs(inp)
        x = self.to_out(x)
        return x

In [39]:
sith_params1 = {"in_features":2, 
                "tau_min":1, "tau_max":50.0, 
                "k":75, 'dt':1,
                "ntau":30, 'g':0.,  
                "ttype":ttype, 
                "hidden_size":30, "act_func":nn.ReLU()
               }
sith_params2 = {"in_features":sith_params1['hidden_size'], 
                "tau_min":1, "tau_max":2000.0, 
                "k":75, 'dt':1,
                "ntau":30, 'g':0., 
                "ttype":ttype, 
                "hidden_size":40, "act_func":nn.ReLU()
                }
sith_params3 = {"in_features":sith_params2['hidden_size'], 
                "tau_min":1, "tau_max":2000.0, 
                "k":75, 'dt':1,
                "ntau":30, 'g':0., 
                "ttype":ttype, 
                "hidden_size":35, "act_func":nn.ReLU()
                }
layer_params = [sith_params1, sith_params2]#, sith_params3]
model = DeepSITH_Predictor(out_features=1,
                           layer_params=layer_params, 
                           dropout=.0).cuda()
print(model)
for i, l in enumerate(model.hs.layers):
    print("Layer {}".format(i), l.sith.tau_star)
tot_weights = 0
for p in model.parameters():
    tot_weights += p.numel()
print("Total Weights:", tot_weights)
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

DeepSITH_Predictor(
  (hs): DeepSITH(
    (layers): ModuleList(
      (0): _DeepSITH_core(
        (sith): iSITH(ntau=30, tau_min=1, tau_max=50.0, buff_max=150.0, dt=1, k=75, g=0.0)
        (linear): Sequential(
          (0): Linear(in_features=60, out_features=30, bias=True)
          (1): ReLU()
        )
      )
      (1): _DeepSITH_core(
        (sith): iSITH(ntau=30, tau_min=1, tau_max=2000.0, buff_max=6000.0, dt=1, k=75, g=0.0)
        (linear): Sequential(
          (0): Linear(in_features=900, out_features=40, bias=True)
          (1): ReLU()
        )
      )
    )
    (dropouts): ModuleList(
      (0): Dropout(p=0.0, inplace=False)
    )
  )
  (to_out): Linear(in_features=40, out_features=1, bias=True)
)
Layer 0 tensor([ 1.0000,  1.1444,  1.3097,  1.4988,  1.7153,  1.9630,  2.2465,  2.5710,
         2.9423,  3.3672,  3.8535,  4.4100,  5.0469,  5.7757,  6.6099,  7.5645,
         8.6569,  9.9071, 11.3379, 12.9753, 14.8492, 16.9937, 19.4479, 22.2566,
        25.4709, 29.1494, 3

In [ ]:
plt.plot(model.hs.layers[2].sith.filters[:,0,0,:].detach().cpu().T);

In [ ]:
seq_length=2000
progress_bar = tqdm(range(int(epochs)), bar_format='{l_bar}{bar:5}{r_bar}{bar:-5b}')
for e in progress_bar:
    train(model, ttype, seq_length,
          optimizer, loss_func, batch_size=10, loss_buffer_size=1000,
          epoch=e, perf_file=join('perf','adding_deepsith_7.csv'),
          prog_bar=progress_bar)